In [3]:
from google.colab import files

files.upload()

Saving fakenews.csv to fakenews.csv


In [39]:
import pandas as pd

data = pd.read_csv("/content/fakenews.csv")

In [40]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#**Preprocessing Data**

In [41]:
import re
from gensim.parsing.preprocessing import remove_stopwords
import string
import unicodedata

def lower_case(text):
    return text.lower()

def remove_punctuation(text):
    text_list = [char for char in text if char not in string.punctuation]
    clean_text = "".join(text_list)
    return clean_text

def remove_numbers(text):
    text_list = []
    for i in text:
        if not re.search("\d", i):
            text_list.append(i)
    return "".join(text_list)

def remove_special_char(text):
    pat =  r'[^a-zA-z0-9]' 
    return re.sub(pat, " ", text)

def remove_square_bracket(text):
    clean_text = re.sub(r'\[[0-9]*\]', ' ', text)
    return clean_text

def remove_white_spaces_char(text):
    clean_text = re.sub(r'\s+', ' ', text)
    
    return clean_text

def remove_unicode_char(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text



def remove_user(text):
    clean_text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    
    return clean_text

def remove_links(text):
    clean_text = re.sub(r'http\S+', '', text)
    clean_text = re.sub(r'bit.ly/\S+', '', text)
    clean_text = clean_text.strip('[link]')
    
    return clean_text

def remove_html(text):
    html = re.compile('<.*?>')
    
    return html.sub(r'',text)

def remove_email(text):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',text)

def remove_stopwords_from_text(text):
    
    clean_text = remove_stopwords(text)
    
    return clean_text




In [42]:
data["title"].head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [43]:
data["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [44]:
data["title"].astype(str)

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 20800, dtype: object

In [45]:
data["clean_title"] = data["title"].astype(str).apply(lower_case)
data["clean_title"] = data["clean_title"].apply(remove_punctuation)
data["clean_title"] = data["clean_title"].apply(remove_numbers)
data["clean_title"] = data["clean_title"].apply(remove_special_char)
data["clean_title"] = data["clean_title"].apply(remove_square_bracket)
data["clean_title"] = data["clean_title"].apply(remove_white_spaces_char)
data["clean_title"] = data["clean_title"].apply(remove_unicode_char)
data["clean_title"] = data["clean_title"].apply(remove_user)
data["clean_title"] = data["clean_title"].apply(remove_links)
data["clean_title"] = data["clean_title"].apply(remove_html)
data["clean_title"] = data["clean_title"].apply(remove_email)
data["clean_title"] = data["clean_title"].apply(remove_stopwords_from_text)

In [46]:
data["clean_title"]

0        house dem aide t comey s letter jason chaffetz...
1         flynn hillary clinton big woman campus breitbart
2                                              truth fired
3             civilians killed single airstrike identified
4        ranian woman jailed fictional unpublished stor...
                               ...                        
20795         rapper ti trump poster child white supremacy
20796    fl playoffs schedule matchups odds new york times
20797    macy s said receive takeover approach hudson s...
20798           ato russia hold parallel exercises balkans
20799                                        keeps f alive
Name: clean_title, Length: 20800, dtype: object

In [47]:
data.head()

,id,title,author,text,label,clean_title
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,house dem aide t comey s letter jason chaffetz...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,flynn hillary clinton big woman campus breitbart
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,truth fired
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,civilians killed single airstrike identified
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,ranian woman jailed fictional unpublished stor...


In [48]:
data[["label", "clean_title"]]

,label,clean_title
0,1,house dem aide t comey s letter jason chaffetz...
1,0,flynn hillary clinton big woman campus breitbart
2,1,truth fired
3,1,civilians killed single airstrike identified
4,1,ranian woman jailed fictional unpublished stor...
...,...,...
20795,0,rapper ti trump poster child white supremacy
20796,0,fl playoffs schedule matchups odds new york times
20797,0,macy s said receive takeover approach hudson s...
20798,1,ato russia hold parallel exercises balkans


In [49]:
data[["label","clean_title"]].isnull().sum()

label          0
clean_title    0
dtype: int64

In [50]:
data[["label","clean_title"]].count()

label          20800
clean_title    20800
dtype: int64

#**Splitting Data**

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
x_train, x_test, y_train, y_test = train_test_split(data["clean_title"].to_numpy(),
                                                    data["label"].to_numpy(),
                                                    test_size = 0.2,
                                                    random_state = 42
                                                    )

In [53]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16640,), (4160,), (16640,), (4160,))

#**Creating a base Model**

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [55]:
base_model = Pipeline([
                       ("tfidf", TfidfVectorizer()),
                       ("clf", MultinomialNB())
])

In [56]:
base_model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [57]:
base_model_score = base_model.score(x_test, y_test)

print(f"Base model Accuracy Score : {base_model_score *100 : .2f} %")

Base model Accuracy Score :  89.21 %


# **LSTM Model**

In [58]:
# finding the average words in a sentence

round(sum([len(i.split()) for i in x_train])/len(x_train))

8

In [59]:
import numpy as np
sen_len = [len(i.split()) for i in x_train]
output_len = int(np.percentile(sen_len, 95))
output_len


13

In [60]:
# total word in sentenses
round(sum([len(i.split()) for i in x_train]))

135455

In [61]:
# import text vectorizer from tensorflow

from tensorflow.keras.layers import TextVectorization

In [62]:
max_token = 21000
average_len = 8
text_vectorizer = TextVectorization(max_tokens= max_token,
                                    output_sequence_length= average_len)

In [63]:
text_vectorizer.adapt(x_train)

In [64]:
# sample text tokenize
sample_texts = "Can I detect fake news on the internet"

text_vectorizer([sample_texts])

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[   1,    1, 9360,   49,   13,    1,    1,  605]])>

In [65]:
text_vectorizer.get_config()

{'batch_input_shape': (None,),
 'dtype': 'string',
 'max_tokens': 21000,
 'name': 'text_vectorization_1',
 'ngrams': None,
 'output_mode': 'int',
 'output_sequence_length': 8,
 'pad_to_max_tokens': False,
 'split': 'whitespace',
 'standardize': 'lower_and_strip_punctuation',
 'trainable': True}

In [66]:
len(text_vectorizer.get_vocabulary())

20451

#**Creating Embedding Layer**

In [67]:
# embedding layer

from tensorflow.keras import layers

input_dim = len(text_vectorizer.get_vocabulary())

embedding = layers.Embedding(input_dim = input_dim,
                             output_dim = 128,
                             )

In [68]:
# sample text embedding

embedding(text_vectorizer([sample_texts]))

<tf.Tensor: shape=(1, 8, 128), dtype=float32, numpy=
array([[[-0.00903221,  0.04827281,  0.03428713, ..., -0.00390363,
         -0.01763769, -0.00456107],
        [-0.00903221,  0.04827281,  0.03428713, ..., -0.00390363,
         -0.01763769, -0.00456107],
        [-0.04830901,  0.03988102,  0.00225858, ...,  0.0132849 ,
          0.01145694,  0.03581959],
        ...,
        [-0.00903221,  0.04827281,  0.03428713, ..., -0.00390363,
         -0.01763769, -0.00456107],
        [-0.00903221,  0.04827281,  0.03428713, ..., -0.00390363,
         -0.01763769, -0.00456107],
        [ 0.04168909, -0.04532553,  0.02998661, ...,  0.03256637,
         -0.04038534,  0.00605749]]], dtype=float32)>

In [79]:
# creating a lstm model

import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape= (1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(150, return_sequences= True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation= "sigmoid")(x)
model_lstm = tf.keras.Model(inputs, outputs, name = "lstm_model")

In [80]:
# compliling the model

model_lstm.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [82]:
model_lstm.summary()

Model: "lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 8)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 8, 128)            2617728   
_________________________________________________________________
lstm_4 (LSTM)                (None, 8, 150)            167400    
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                55040     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 2,840,233
Trainable params: 2,840,233
Non-trainable params: 0
______________________________________________

In [83]:
# fitting the model

model_lstm.fit(x_train, y_train,
               epochs = 10,
               validation_data = (x_test, y_test)
               )

Epoch 1/10
520/520 [==============================] - 13s 17ms/step - loss: 0.0333 - accuracy: 0.9918 - val_loss: 0.6950 - val_accuracy: 0.8514
Epoch 2/10
520/520 [==============================] - 8s 15ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.8287 - val_accuracy: 0.8603
Epoch 3/10
520/520 [==============================] - 8s 15ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.8214 - val_accuracy: 0.8591
Epoch 4/10
520/520 [==============================] - 8s 15ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.6692 - val_accuracy: 0.8596
Epoch 5/10
520/520 [==============================] - 8s 15ms/step - loss: 4.2493e-04 - accuracy: 0.9999 - val_loss: 0.9857 - val_accuracy: 0.8560
Epoch 6/10
520/520 [==============================] - 8s 15ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 0.7221 - val_accuracy: 0.8526
Epoch 7/10
520/520 [==============================] - 8s 15ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 0.7789 - val_accuracy: 0

In [84]:
lstm_model_evaluate = model_lstm.evaluate(x_test, y_test)

print(f"Model Accuracy Score : {lstm_model_evaluate[1] * 100 : .2f} %")

130/130 [==============================] - 1s 5ms/step - loss: 1.0122 - accuracy: 0.8579
Model Accuracy Score :  85.79 %
